In [ ]:
import openseespy.opensees as op
import numpy as np
import vfo.vfo as vfo 
import math 
import opsvis as opsv
%matplotlib notebook
import matplotlib.pyplot as plt

# Define model and materials 

In [ ]:
#units used are kN,m and second. 

#model
#remove existing model 
op.wipe()

#set a model 
#model('basic', '-ndm', ndm, '-ndf')
op.model('basic', '-ndm', 2, '-ndf', 3)

#define concrete material
fpc = -30.*10**3      #M30 concrete
fpcu = fpc*0.1
epsc0 = -0.002
epsU = epsc0*10
lam = 0.2
ft = -fpc/30
Ets = 2*fpc / (epsc0 * 20)
#uniaxialMaterial('Concrete02', matTag, fpc, epsc0, fpcu, epsU, lambda, ft, Ets)
op.uniaxialMaterial('Concrete02', 1, fpc, epsc0, fpcu, epsU, lam, ft, Ets)

#define reinforcing steel material 
Fy = 500.*10**3         #Fe500 steel 
E0 = 210.*10**6         #youngs modulus of 210 Gpa
b = 0.02
#uniaxialMaterial('Steel02', matTag, Fy, E0, b)
op.uniaxialMaterial('Steel02', 2, Fy, E0, b)


# Create nodes

In [ ]:
#create node
number_storey=4
number_bays=2
storey_height=3
bay_width=5        # 5 meters
number_nodes=(number_storey+1)*(number_bays+1)
nodex=np.array([])
for i in range(number_bays+1):
    nodex=np.append(nodex,i*bay_width)
nodey=np.array([])
for i in range(number_storey+1):
    nodey=np.append(nodey,i*storey_height)

node_tag=np.array([])
for counter,i in enumerate(nodex):
    if counter==0:
        tag=1
    else:
        tag=1+counter
    for j in nodey:
        op.node(tag,i,j)
        node_tag=np.append(node_tag,tag)
        tag=tag+3
    

# set boundary condition
op.fix(1, 1, 1, 1)
op.fix(2, 1, 1, 1)
op.fix(3, 1, 1, 1)

#print(node_tag)

## define beam section and add beam elements

In [ ]:
#define beam section
# Get Verticies
h = 0.6     #600mm
w = 0.4     #400mm
vertices = np.array([-h/2, w/2, -h/2, -w/2, h/2, -w/2, h/2, w/2])

# Define Rebar Info
rebarZ = np.array([-0.15, 0, 0.15])
rebarY = np.array([-0.225, -0.15, 0.15, 0.225])
Abar = np.pi*(0.03/2)**2                                    # 30mm diameter bars

Nbar = len(rebarZ)*len(rebarY)
rebarYZ = np.zeros([Nbar,2])

for ii, Y in enumerate(rebarY):
    for jj, Z in enumerate(rebarZ):
        rebarYZ[ii*len(rebarZ) + jj, :] = [Y, Z]

NfibeZ = 1
NfibeY = 50       
gamma=25
mass_beam=25*h*w
# Define Sections
#  section('Fiber', secTag)
op.section('Fiber', 1)

#  patch('quad', matTag, numSubdivIJ, numSubdivJK, *crdsI, *crdsJ, *crdsK, *crdsL)
op.patch('quad', 1, NfibeZ, NfibeY, *vertices)

for YZ in rebarYZ:       
    #  fiber(yloc, zloc, A, matTag)
    op.fiber(*YZ, Abar, 2)


# Define transform and integration
op.geomTransf('Linear', 1)
op.geomTransf('PDelta', 2)

#  beamIntegration('Lobatto', tag, secTag, N)
op.beamIntegration('Lobatto', 1, 1, 4)
#op.beamIntegration('Lobatto', 2, 2, 4)
                                                       


#beam elements
element_tag_beams=np.array([])
tag=1
for i in range(number_nodes+1):
    if i==0:
        continue
    else:
        if i<4:
            continue
        else:
            if (i%3)==0:
                continue
            else:
                #element('forceBeamColumn', eleTag, *eleNodes, transfTag, integrationTag, '-iter', maxIter=10, tol=1e-12,'-mass', mass=0.0))
                op.element('forceBeamColumn', tag,*[i,i+1] , 1, 1, '-iter', 30, 1e-12,'-mass', mass_beam)
                element_tag_beams=np.append(element_tag_beams,tag)
    tag=tag+1

print(element_tag_beams)

## define column section and add column elements

In [ ]:
# Define cross-section for nonlinear columns
# ------------------------------------------

#  some parameters
h = 0.65       #650mm
w = 0.65       #650mm
vertices_column = np.array([-h/2, w/2, -h/2, -w/2, h/2, -w/2, h/2, w/2])

cover = 0.06     #60mm
As = np.pi*(0.03/2)**2

# some variables derived from the parameters
y1 = h / 2.0
z1 = w / 2.0
mass_column=gamma*h*w
#section('Fiber', secTag)
op.section('Fiber', 2)

# Create the concrete  fibers
# patch('quad', matTag, numSubdivIJ, numSubdivJK, *crdsI, *crdsJ, *crdsK, *crdsL)
op.patch('quad', 1, 1, 10, *vertices_column)

# Create the reinforcing fibers (left, middle, right)
#layer('straight', matTag, numFiber, areaFiber, *start, *end)
op.layer('straight', 2, 3, As, y1 - cover, z1 - cover, y1 - cover, cover - z1)
op.layer('straight', 2, 2, As, 0.0, z1 - cover, 0.0, cover - z1)
op.layer('straight', 2, 3, As, cover - y1, z1 - cover, cover - y1, cover - z1)

# Define column elements
# ----------------------

# Geometry of column element

op.geomTransf('Linear', 3)

# Number of integration points along length of element
np = 5

# Lobatto integratoin
#  beamIntegration('Lobatto', tag, secTag, N)
op.beamIntegration('Lobatto', 3, 2, np)

#element_tag_columns=np.array([])
# Create the coulumns using Beam-column elements
for i in range(number_nodes+1):
    if i==0:
        continue
    else:
        if i>=13:
            break 
        else:
            #element('forceBeamColumn', eleTag, *eleNodes, transfTag, integrationTag, '-iter', maxIter=10, tol=1e-12,'-mass', mass=0.0))
            op.element('forceBeamColumn', tag,*[i,i+3] , 3, 3, '-iter', 30, 1e-12,'-mass',mass_column)
            #element_tag_columns=np.append(element_tag_columns,tag)
            
    tag=tag+1
        

#print(element_tag_columns)        
#vfo.plot_model(show_nodetags="yes")

## Gravity loads

In [ ]:
#gravity loads
#create a linear time series object 
op.timeSeries('Linear', 1)

#create a plain load pattern
op.pattern('Plain', 1, 1)

#apply UDL to the beams 
WY= -100 #KN/m
op.eleLoad('-ele', *element_tag_beams, '-range', 1, 8, '-type', '-beamUniform', WY)

# Create the nodal load - command: load nodeID xForce yForce
#op.load(4, 0, WY, 0)

#opsv.plot_loads_2d(nep=5)

## Gravity analysis 

In [ ]:
#create record files 
analysis_name='My_2D_project_gravity'
#Record results
op.recorder('Node','-file',analysis_name + '_NodeDisp.out','-time','-node',*node_tag,'-dof',1,2,3,'disp')
op.recorder('Node','-file',analysis_name +'_Reaction.out','-time','-node',3,'-dof',1,2,3,'reaction')
op.recorder('Element','-file',analysis_name +'_Elements.out','-time','-ele',*element_tag_beams,'-dof',1,2,3,'forces')    


#analysis terminologies 
# create SOE
op.system("BandGeneral")

# create DOF number
op.numberer("RCM")

# create constraint handler
op.constraints("Transformation")

# create integrator
op.integrator("LoadControl", 0.1)

# Create the convergence test, the norm of the residual with a tolerance of
# 1e-12 and a max number of iterations of 10
op.test('NormDispIncr', 1.0e-7, 30)

# create algorithm
op.algorithm("Newton")

# create analysis object
op.analysis("Static")

# perform the analysis
op.analyze(10)

 



## Dynamic load and dynamic analysis 

In [ ]:
#dynamic loads 
#create a path time series object 


#timeSeries('Path', tag, '-dt', dt=0.0, '-values', *values, '-time', *time, '-filePath', filePath='', '-fileTime', fileTime='', '-factor', factor=1.0, '-startTime', startTime=0.0, '-useLast', '-prependZero')
op.timeSeries('Path', 2, '-dt', 0.01, '-filePath', 'BM68elc.acc', '-factor', 1.0)
op.pattern('UniformExcitation', 2, 1, '-accel', 2)

analysis_name='My_2D_project_dynamic'
#Record results
op.recorder('Node','-file',analysis_name + '_NodeDisp.out','-time','-node',15,'-dof',1,'disp')

#generate eigen values 
eigen = op. eigen('-fullGenLapack',4)
#op.modalProperties('-print', '-file', 'ModalReport.txt', '-unorm')


print(eigen)
import math
power = math.pow(eigen[0], 0.5)
betaKcomm = 2 * (0.02/power)
print(betaKcomm)
op.rayleigh(0.0, 0.0, 0.0, betaKcomm)

op.wipeAnalysis()
op.constraints('Plain')
op.numberer('Plain')
op.system('BandGeneral')
op.test('NormDispIncr', 1e-8, 1000)
op.algorithm('KrylovNewton')
op.integrator('Newmark', 0.5, 0.25)
op.analysis('Transient')
#op.analyze(1000, 0.0002)

u3 = op.nodeDisp(15)
print("u2 = ", u3)




## Run an animation

In [ ]:
import numpy as np
el_tags = op.getEleTags()
nels = len(el_tags)
n_steps=1000
Eds = np.zeros((n_steps, nels, 6))
timeV = np.zeros(n_steps)
dt=0.005
# transient analysis loop and collecting the data
for step in range(n_steps):
    op.analyze(1, dt)
    timeV[step] = op.getTime()
    # collect disp for element nodes
    for el_i, ele_tag in enumerate(el_tags):
        nd1, nd2 = op.eleNodes(ele_tag)
        Eds[step, el_i, :] = [op.nodeDisp(nd1)[0],
                              op.nodeDisp(nd1)[1],
                              op.nodeDisp(nd1)[2],
                              op.nodeDisp(nd2)[0],
                              op.nodeDisp(nd2)[1],
                              op.nodeDisp(nd2)[2]]

input_parameters = (20.8, 300., 8.)
pf, sfac_a, tkt = input_parameters

fmt_defo = {'color': 'blue', 'linestyle': 'solid', 'linewidth': 3.0,
            'marker': '', 'markersize': 6}
anim = opsv.anim_defo(Eds, timeV,sfac_a, fmt_defo=fmt_defo,
                      xlim=[-1, 12], ylim=[-3, 16], fig_wi_he=(20., 19.))

#plt.show()
anim.save('dynamic_images.gif',writer='pillow')
op.wipe()